<a href="https://colab.research.google.com/github/niamh-m/Dental-Magic-Employees/blob/main/dMiningCA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Libraries
import numpy as np
import pandas as pd
import imblearn
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
%matplotlib inline


# Set random seed
np.random.seed(0)

In [ ]:
#@title Import Data
path = "/DentalMagicEmployees.csv"
df= pd.read_csv(path)
#df.head()
df.describe()

,Zip,Age,Pay,Employee Number
count,299.000000,299.000000,299.000000,2.990000e+02
mean,6735.735786,38.511706,30.750502,1.204420e+09
std,17220.951984,8.943279,15.261313,1.828106e+08
min,1013.000000,25.000000,14.000000,6.020003e+08
25%,1895.500000,31.000000,20.000000,1.102024e+09
50%,2132.000000,37.000000,24.000000,1.203032e+09
75%,2433.000000,44.000000,43.000000,1.401065e+09
max,98052.000000,67.000000,80.000000,1.988300e+09


In [ ]:
df.shape

(299, 22)

In [ ]:
df.head()

,Employee Name,State,Zip,DOB,Age,Sex,MaritalDesc,CitizenDesc,Hispanic/Latino,RaceDesc,Date of Hire,Date of Termination,Reason For Term,Department,Position,Pay,Manager,Source,Performance,Employee Number,Employment Status,Active Status
0,"Brown, Mia",MA,1450,24/11/1985 00:00,32,Female,Married,US Citizen,No,Black or African American,27/10/2008 00:00,NaN,N/A - still employed,Admin Offices,Accountant I,28.5,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets,1103024456,Active,True
1,"LaRotonda, William",MA,1460,26/04/1984 00:00,33,Male,Divorced,US Citizen,No,Black or African American,06/01/2014 00:00,NaN,N/A - still employed,Admin Offices,Accountant I,23.0,Brandon R. LeBlanc,Website Banner Ads,Fully Meets,1106026572,Active,True
2,"Steans, Tyrone",MA,2703,01/09/1986 00:00,31,Male,Single,US Citizen,No,White,29/09/2014 00:00,NaN,N/A - still employed,Admin Offices,Accountant I,29.0,Brandon R. LeBlanc,Internet Search,Fully Meets,1302053333,Active,True
3,"Howard, Estelle",MA,2170,16/09/1985 00:00,32,Female,Married,US Citizen,No,White,16/02/2015 00:00,NaN,N/A - still employed,Admin Offices,Administrative Assistant,21.5,Brandon R. LeBlanc,Pay Per Click - Google,N/A- too early to review,1211050782,Active,True
4,"Singh, Nan",MA,2330,19/05/1988 00:00,29,Female,Single,US Citizen,No,White,01/05/2015 00:00,NaN,N/A - still employed,Admin Offices,Administrative Assistant,16.6,Brandon R. LeBlanc,Website Banner Ads,N/A- too early to review,1307059817,Active,True


In [ ]:
#@title Generate Employee source subtype column
dict = {"Billboard" : 'External Ad', "MBTA ads" : 'External Ad',"Newspager/Magazine" : 'External Ad',"Company Intranet - Partner" : 'Internal',
        "On-line Web application" : 'Internal',"Employee Referral" : 'Internal',"Social Networks - Facebook Twitter etc" : 'Google or SocMed',
        "Internet Search" : 'Google or SocMed',"Pay Per Click" : 'Google or SocMed',"Pay Per Click - Google" : 'Google or SocMed',
        "Search Engine - Google Bing Yahoo" : 'Google or SocMed',"Diversity Job Fair" : 'Recruitment Fair',
        "Information Session" : 'Recruitment Fair',"On-campus Recruiting" : 'Recruitment Fair',"Professional Society" : 'Recruitment Fair',
        "Monster.com" : 'Jobs Website',"Glassdoor" : 'Jobs Website',"Careerbuilder" : 'Jobs Website',
        "Other" : 'Other',"Vendor Referral" : 'Other',"Website Banner Ads" : 'Other',"Word of Mouth" : 'Other'}
df2=df.replace({"Employee Source": dict})

df2.to_csv('out.csv', index=False)


##### CHECKED DF2 IN RAPIDMINER FOR INCREASE IN MODEL PERFRORMANCE => NO INCREASE SO WILL USE ORIGINAL DF #####

In [ ]:
#@title Separate feature variables and target variable
X = df.drop(['Employee Name','Employee Number', 'State','Zip', 'DOB', 'CitizenDesc','Hispanic/Latino',
             'RaceDesc','Date of Hire','Date of Termination','Reason For Term','Employment Status','Active Status'],axis=1)

#### Now X only has the columns we need ####

y = df['Active Status']


y.head()


0    True
1    True
2    True
3    True
4    True
Name: Active Status, dtype: bool

In [ ]:
y.shape


(299,)

In [ ]:
X.shape

(299, 9)

In [ ]:
print(X['Sex'].unique())

['Female' 'Male']


In [ ]:
X['Sex'].replace('Female', 1,inplace=True)
X['Sex'].replace('Male', 0,inplace=True)
print(X['Sex'].unique())

[1 0]


In [ ]:
print(X['Position'].unique())

['Accountant I' 'Administrative Assistant' 'Shared Services Manager'
 'Sr. Accountant' 'President & CEO' 'CIO' 'Database Administrator'
 'IT Director' 'IT Manager - DB' 'IT Manager - Infra'
 'IT Manager - Support' 'IT Support' 'Network Engineer' 'Sr. DBA'
 'Sr. Network Engineer' 'Director of Operations' 'Production Manager'
 'Production Technician I' 'Production Technician II' 'Area Sales Manager'
 'Director of Sales' 'Sales Manager' 'Software Engineer'
 'Software Engineering Manager']


In [ ]:
#@title Balance target variable
# check version number
import imblearn
print(imblearn.__version__)

# Exploring the target variable
fig = px.histogram(df,x='Active Status')
fig.show()



0.8.1


In [ ]:
#@title Ignore
# model = GradientBoostingClassifier(loss='deviance', # deviance or explonential
#                                criterion='mse', # {‘friedman_mse’, ‘mse’, ‘mae’}, default=’friedman_mse’
#                                learning_rate=0.01, # default 0.1
#                                subsample=1.0, # default=1.0
#                                random_state=0, # random state for reproducibility
#                                max_features=None, # number of random features to use sqrt(n_features), default=None
#                                min_samples_leaf=1, # minimum no of observarions allowed in a leaf
#                                max_depth=2, # maximum depth of the tree, default=3
#                                n_estimators=3 # how many trees to build
#                               )


In [ ]:
#@title check Datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Employee Name        299 non-null    object 
 1   State                299 non-null    object 
 2   Zip                  299 non-null    int64  
 3   DOB                  299 non-null    object 
 4   Age                  299 non-null    int64  
 5   Sex                  299 non-null    object 
 6   MaritalDesc          299 non-null    object 
 7   CitizenDesc          299 non-null    object 
 8   Hispanic/Latino      299 non-null    object 
 9   RaceDesc             299 non-null    object 
 10  Date of Hire         299 non-null    object 
 11  Date of Termination  102 non-null    object 
 12  Reason For Term      299 non-null    object 
 13  Department           299 non-null    object 
 14  Position             299 non-null    object 
 15  Pay                  299 non-null    flo

In [ ]:
from sklearn import preprocessing
def convert(X):
    number = preprocessing.LabelEncoder()
    X['Source'] = number.fit_transform(X.Source)
    X['MaritalDesc'] = number.fit_transform(X.MaritalDesc)
    X['Department'] = number.fit_transform(X.Department)
    X['Position'] = number.fit_transform(X.Position)
    X['Manager'] = number.fit_transform(X.Manager)
    X['Performance'] = number.fit_transform(X.Performance)
    X=X.fillna(-999)
    return X

X=convert(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(209, 9)
(90, 9)
(209,)
(90,)


In [ ]:
#@title Sampling

from imblearn.under_sampling import RandomUnderSampler

under_sampler = RandomUnderSampler(random_state=42)
X_res, y_res = under_sampler.fit_resample(X_train, y_train)
X_train=X_res
y_train=y_res

In [ ]:
# Machine Learning Models
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

################################################################################


from sklearn.metrics import classification_report, confusion_matrix

# Takes in a model, trains the model, and evaluates the model on the test set
def fit_and_evaluate(model):
    
    # Train the model
    model.fit(X, y)
    
    # Make predictions and evalute
    predictions = model.predict(X_test)
    
    # Return the performance metric
    return print(classification_report(y_test, predictions), '\n', confusion_matrix(y_test, predictions))

In [ ]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, y)


In [ ]:
#tree.plot_tree(clf)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Takes in a model, trains the model, and evaluates the model on the test set
def fit_and_evaluate(model):
    
    # Train the model
    model.fit(X, y)
    
    # Make predictions and evalute
    predictions = model.predict(X_test)
    
    # Return the performance metric
    return print(classification_report(y_test, predictions), '\n', confusion_matrix(y_test, predictions))

In [ ]:
lr = LogisticRegression()

fit_and_evaluate(lr)

              precision    recall  f1-score   support

       False       0.50      0.15      0.23        33
        True       0.65      0.91      0.76        57

    accuracy                           0.63        90
   macro avg       0.57      0.53      0.50        90
weighted avg       0.60      0.63      0.57        90
 
 [[ 5 28]
 [ 5 52]]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [ ]:
svm = SVC(C = 1000, gamma = 0.1)

fit_and_evaluate(svm)


##### LIKELY OVERFITTING #####

              precision    recall  f1-score   support

       False       1.00      1.00      1.00        33
        True       1.00      1.00      1.00        57

    accuracy                           1.00        90
   macro avg       1.00      1.00      1.00        90
weighted avg       1.00      1.00      1.00        90
 
 [[33  0]
 [ 0 57]]


In [ ]:
gradient_boosted = GradientBoostingClassifier(random_state=60)

fit_and_evaluate(gradient_boosted)

              precision    recall  f1-score   support

       False       1.00      0.91      0.95        33
        True       0.95      1.00      0.97        57

    accuracy                           0.97        90
   macro avg       0.97      0.95      0.96        90
weighted avg       0.97      0.97      0.97        90
 
 [[30  3]
 [ 0 57]]


In [ ]:
#@title Important Features
importances = gradient_boosted.feature_importances_

feature_list = list(X.columns)

feature_results = pd.DataFrame({'feature': feature_list,
                               'importance': importances})

feature_results = feature_results.sort_values(by=['importance'], ascending=False)

feature_results

,feature,importance
5,Pay,0.258255
7,Source,0.197588
0,Age,0.142827
6,Manager,0.128358
2,MaritalDesc,0.104863
8,Performance,0.076723
4,Position,0.072312
3,Department,0.015937
1,Sex,0.003136
